# Vehicle Routing Problem (VRP) Genetic Algorithm

This notebook demonstrates a complete VRP solution using genetic algorithms with the DEAP library.
The project is organized into modular components for better maintainability and reusability.

## 1. Import Required Modules

In [ ]:
# Import project modules
from locations import generate_random_locations, create_depot
from vrp_solver import VRPSolver
from tuning import VRPTuner, compare_tuning_methods
from plotting import plot_vrp_routes, plot_tuning_results, plot_parameter_correlation, create_vrp_summary_dashboard
from config import DEFAULT_VRP_CONFIG, DEFAULT_GA_CONFIG, TUNING_RANGES

# Import other required libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import random

print("All modules imported successfully!")

## 2. Setup VRP Problem

In [ ]:
# Generate random locations
num_locations = DEFAULT_VRP_CONFIG["num_locations"]
num_vehicles = DEFAULT_VRP_CONFIG["num_vehicles"]

locations = generate_random_locations(
    num_locations=num_locations,
    x_range=DEFAULT_VRP_CONFIG["location_range_x"],
    y_range=DEFAULT_VRP_CONFIG["location_range_y"],
    seed=DEFAULT_VRP_CONFIG["seed"]
)

depot = create_depot(*DEFAULT_VRP_CONFIG["depot"])

print(f"Generated {num_locations} locations")
print(f"Depot location: {depot}")
print(f"Number of vehicles: {num_vehicles}")

## 3. Initialize VRP Solver

In [ ]:
# Create VRP solver instance
solver = VRPSolver(locations, depot, num_vehicles)
print("VRP Solver initialized!")

## 4. Solve VRP with Default Parameters

In [ ]:
# Solve with default parameters
print("Solving VRP with default parameters...")
best_individual, population, stats, hof = solver.solve(
    pop_size=DEFAULT_GA_CONFIG["pop_size"],
    cxpb=DEFAULT_GA_CONFIG["cxpb"],
    mutpb=DEFAULT_GA_CONFIG["mutpb"],
    ngen=50,  # Reduced for demo
    tournsize=DEFAULT_GA_CONFIG["tournsize"],
    seed=DEFAULT_GA_CONFIG["seed"]
)

print(f"Best distance found: {best_individual.fitness.values[0]:.2f}")
print(f"Balance penalty: {best_individual.fitness.values[1]:.2f}")

## 5. Visualize Default Solution

In [ ]:
# Plot the solution
fig = plot_vrp_routes(locations, depot, best_individual, num_vehicles, 
                     title="VRP Solution with Default Parameters")
fig.show()

## 6. Parameter Tuning

In [ ]:
# Initialize tuner
tuner = VRPTuner(locations, depot, num_vehicles)

# Perform random search tuning
print("Starting parameter tuning...")
best_params, best_tuned_individual, best_distance, tuning_results = tuner.random_search_tuning(
    n_trials=10,  # Reduced for demo
    param_ranges=TUNING_RANGES,
    ngen=20
)

## 7. Visualize Tuned Solution

In [ ]:
# Plot the tuned solution
fig = plot_vrp_routes(locations, depot, best_tuned_individual, num_vehicles, 
                     title=f"VRP Solution with Tuned Parameters (Distance: {best_distance:.1f})")
fig.show()

## 8. Compare Results

In [ ]:
# Compare default vs tuned results
default_distance = best_individual.fitness.values[0]
tuned_distance = best_distance
improvement = ((default_distance - tuned_distance) / default_distance) * 100

print("=== RESULTS COMPARISON ===")
print(f"Default GA Distance: {default_distance:.2f}")
print(f"Tuned GA Distance: {tuned_distance:.2f}")
print(f"Improvement: {improvement:.2f}%")
print(f"Best Parameters: {best_params}")

## 9. Tuning Results Visualization

In [ ]:
# Plot tuning progress
fig = plot_tuning_results(tuning_results)
fig.show()

In [ ]:
# Plot parameter correlation
fig = plot_parameter_correlation(tuning_results)
fig.show()

## 10. Comprehensive Dashboard

In [ ]:
# Create comprehensive dashboard
dashboard = create_vrp_summary_dashboard(
    locations, depot, best_tuned_individual, num_vehicles, tuning_results
)
dashboard.show()

## 11. Save Results

In [ ]:
# Save tuning results to CSV
results_df = pd.DataFrame(tuning_results)
results_df.to_csv('tuning_results.csv', index=False)

# Save best solution
best_solution = {
    'best_individual': list(best_tuned_individual),
    'best_distance': best_distance,
    'best_params': best_params,
    'vehicle_routes': solver.get_vehicle_routes(best_tuned_individual)
}

import json
with open('best_solution.json', 'w') as f:
    json.dump(best_solution, f, indent=2)

print("Results saved to files!")

## 12. Advanced Analysis (Optional)

In [ ]:
# Analyze vehicle route statistics
routes = solver.get_vehicle_routes(best_tuned_individual)

route_stats = []
for i, route in enumerate(routes):
    distance = sum(np.linalg.norm(np.array(route[j+1]) - np.array(route[j])) 
                  for j in range(len(route)-1))
    num_stops = len(route) - 2  # Exclude depot start and end
    route_stats.append({
        'vehicle': i + 1,
        'distance': distance,
        'num_stops': num_stops,
        'avg_distance_per_stop': distance / max(num_stops, 1)
    })

stats_df = pd.DataFrame(route_stats)
print("Vehicle Route Statistics:")
print(stats_df)